In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pdb

In [2]:
start_cost = 3 * 1000 * 16 * 7.6
r = 0.05

def R(ss_t):
    return (ss_t - 5) * 1000 * 16 * 7.6

def Profit(s0, ss0, ss1, upper, lower):
    idx_off = (s0 == 0)
    idx_on = (s0 == 1)

    turn_on = idx_off * (ss0 > upper)
    turn_off = idx_on * (ss0 < lower)

    profit = R(ss1)
    profit[turn_on] = - start_cost
    profit[idx_off & (ss0 <= upper)] = 0

    action = idx_off * (ss0 > upper) + idx_on * (ss0 >= lower)

    return profit, action #action=s1

def valuation(power, gas, upper, lower):#power & gas(10000*97);upper & lower(96)
    ss = (power - 12 * gas)
    n = ss.shape[0]
    N = 2 * 12 * 4 #96
    s = np.zeros((n,N+1))
    V = 0
    for i in range(N): #0-95
        profit, s[:, i+1] = Profit(s[:,i], ss[:,i], ss[:,i+1], upper[i], lower[i])
        V += profit * np.exp(-r*(i+1)/48)
    return np.mean(V)

In [3]:
def cali(Power, Gas, ll = -0.4*10**7, uu = 1.2*10**7, d=10**6):
    V = []
    bound_list = np.array([ll + d * (x+1) for x in range(int((uu -ll)/d))])
    combo = [(i,j) for i in bound_list for j in bound_list]
    for i in bound_list :
        for j in bound_list :
            upper = np.ones(96)*i
            lower = np.ones(96)*j
            V.append(valuation(Power, Gas, upper, lower))
    return combo[np.argmax(V)], np.max(V)

In [4]:
Gas = pd.read_csv('Gas_Simulation_97.csv', header=None).T.values
Power = pd.read_csv('Power_Simulation_97.csv', header=None).T.values

In [5]:
cali(Power, Gas)

((0.0, 0.0), 94779084.74884312)